In [1]:
asv_file = open("asv.txt","r")

In [2]:
import pandas as pd
import re

map_data = pd.read_csv('map.txt',sep='\t')
map_data



,id,linker,primer,barcode
0,gut1,AGAT,TAATACGATTAC,GACCATAG
1,gut2,AGAT,TAATACGATTAC,CCTAGTCC
2,gut3,AGAT,TAATACGATTAC,TACCAGTA
3,gut4,AGAT,TAATACGATTAC,GGGATCTA
4,gut5,AGAT,TAATACGATTAC,ATTAAAAA


Stripping the file to get clean dat

In [3]:
from collections import defaultdict

fasta = defaultdict(list)
with open("asv.txt","r") as file_one:
    for line in file_one:
        if line.startswith(">"):
            fasta[line.strip(">\n")].append(next(file_one).rstrip()[16:])
print(fasta)

defaultdict(<class 'list'>, {' QDQeEODeLV': ['GACCATAGTTGCAGTTGATACTGGCTGTCTTGAGTACAGTAGAGGTGGGCGGAATTCGTGGTGTAGCGGTGAAATGCT'], ' VRXqIfAfMI': ['GGGATCTATTGCAGTTGATACTGGCTGTCTTGAGTACAGTAGAGGTGGGCGGAATTCGTGGTGTAGCGGTGAAATGCT'], ' kDVDCSVoTo': ['CCTAGTCCTTGCAGTTGATACTGGCTGTCTTGAGTACAGTAGAGGTGGGCGGAATTCGTGGTGTAGCGGTGAAATGCT'], ' StPhcMzOZj': ['ATTAAAAACAGCAGCCGCGGTAATACGTAGGTGGCAAGCGTTGTCCGGATTTACTGGGCGTAAAGGATGCGTAGGCGG'], ' ObylTFJkZk': ['TACCAGTACAGCAGCCGCGGTAATACGTAGGTGGCAAGCGTTGTCCGGATTTACTGGGCGTAAAGGATGCGTAGGCGG'], ' ggwPzVXWBC': ['ATTAAAAAGTGTGTGAAAGGCGCGGGCTCAACCCGCGGACGGCACATGATACTGCGAGACTAGAGTAATGGAGGGGTA'], ' zAuautYPfe': ['CCTAGTCCCAGCAGCCGCGGTAATACGTAGGTGGCAAGCGTTGTCCGGATTTACTGGGCGTAAAGGATGCGTAGGCGG'], ' fbBukiSuRh': ['GGGATCTAGTGTGTGAAAGGCGCGGGCTCAACCCGCGGACGGCACATGATACTGCGAGACTAGAGTAATGGAGGGGTA'], ' JqamcXGLgu': ['TACCAGTACAGCAGCCGCGGTAATACGTAGGTGGCAAGCGTTGTCCGGATTTACTGGGCGTAAAGGATGCGTAGGCGG'], ' cgfFzMzsSG': ['CCTAGTCCCAGCAGCCGCGGTAATACGTAGGTGGCAAGCGTTGTCCGGATTTACTGGGCGTAA

Reading the fasta file type
Line that starts with > is the header
Next line is the sequence
find the barcode from the sequence (it matches with all the list of barcodes)

In [4]:
with open("asv.txt","r") as asv_file:
    sequence = []
    for line in asv_file:
        if line.startswith('>'):
            header=line
        else:
            sequence = line 
            for i in range(len(map_data)):
                r = re.findall(map_data['barcode'][i],sequence)
                print(r)
        print("Going to next sequence")               

Going to next sequence
['GACCATAG']
[]
[]
[]
[]
Going to next sequence
Going to next sequence
[]
[]
[]
['GGGATCTA']
[]
Going to next sequence
Going to next sequence
[]
['CCTAGTCC']
[]
[]
[]
Going to next sequence
Going to next sequence
[]
[]
[]
[]
['ATTAAAAA']
Going to next sequence
Going to next sequence
[]
[]
['TACCAGTA']
[]
[]
Going to next sequence
Going to next sequence
[]
[]
[]
[]
['ATTAAAAA']
Going to next sequence
Going to next sequence
[]
['CCTAGTCC']
[]
[]
[]
Going to next sequence
Going to next sequence
[]
[]
[]
['GGGATCTA']
[]
Going to next sequence
Going to next sequence
[]
[]
['TACCAGTA']
[]
[]
Going to next sequence
Going to next sequence
[]
['CCTAGTCC']
[]
[]
[]
Going to next sequence
Going to next sequence
[]
[]
[]
['GGGATCTA']
[]
Going to next sequence
Going to next sequence
[]
[]
['TACCAGTA']
[]
[]
Going to next sequence
Going to next sequence
[]
[]
[]
['GGGATCTA']
[]
Going to next sequence
Going to next sequence
['GACCATAG']
[]
[]
[]
[]
Going to next sequence
Going 

Make sure to: (1) strip out the linkder (adaptor) and primers (2) Use the barcodes to determine which sequence belongs to what sample (and remove the barcode).

Create an output fasta file 
with header line 
> QDQeEODeLV  barcode='GACCATAG' sample=gut1

TTGCAGTTGATACTGGCTGTCTTGAGTACAG...

> VRXqIfAfMI barcode='GGGATCTA' sample=gut4

TTGCAGTTGATACTGGCTGTCTTGAGTACAG...

In [18]:
my_out_file = open("out_fasta.txt","w")
dedup_records = defaultdict(list)
for id,seq in fasta.items():
   for i in range(len(map_data)):
        if(seq[0].find(map_data['barcode'][i])!=-1):
            a = ">"+id+" "+"barcode="+map_data['barcode'][i]+" sample="+map_data['id'][i]+"\n"
            my_out_file.write(a)
            my_out_file.write(seq[0][len(map_data['barcode']):])
            my_out_file.write("\n")
            dedup_records[seq[0][len(map_data['barcode']):]].append(map_data['id'][i])

In [6]:
with open("Output.fasta", 'w') as output:
   for id,seq in fasta.items():
        # Join the ids and write them out as the fasta
        output.write(">{}\n".format('|'.join(id)))
        output.write(seq + "\n")

TypeError: can only concatenate list (not "str") to list

To find unique sequences used them as dictionary keys
and the value is what sample the sequence belongs to

In [16]:
from Bio.SeqIO.FastaIO import SimpleFastaParser
from collections import defaultdict

out_file = open('Output.fasta','r')
dedup_records = defaultdict(list)
for (name, seq) in SimpleFastaParser(out_file):
    # Use the sequence as the key and then have a list of id's as the value
    dedup_records[seq].append(name)

In [19]:
dedup_records

defaultdict(list,
            {'TAGTTGCAGTTGATACTGGCTGTCTTGAGTACAGTAGAGGTGGGCGGAATTCGTGGTGTAGCGGTGAAATGCT': ['gut1',
              'gut1',
              'gut1',
              'gut1'],
             'CTATTGCAGTTGATACTGGCTGTCTTGAGTACAGTAGAGGTGGGCGGAATTCGTGGTGTAGCGGTGAAATGCT': ['gut4',
              'gut4',
              'gut4'],
             'TCCTTGCAGTTGATACTGGCTGTCTTGAGTACAGTAGAGGTGGGCGGAATTCGTGGTGTAGCGGTGAAATGCT': ['gut2'],
             'AAACAGCAGCCGCGGTAATACGTAGGTGGCAAGCGTTGTCCGGATTTACTGGGCGTAAAGGATGCGTAGGCGG': ['gut5',
              'gut5'],
             'GTACAGCAGCCGCGGTAATACGTAGGTGGCAAGCGTTGTCCGGATTTACTGGGCGTAAAGGATGCGTAGGCGG': ['gut3',
              'gut3',
              'gut3',
              'gut3'],
             'AAAGTGTGTGAAAGGCGCGGGCTCAACCCGCGGACGGCACATGATACTGCGAGACTAGAGTAATGGAGGGGTA': ['gut5'],
             'TCCCAGCAGCCGCGGTAATACGTAGGTGGCAAGCGTTGTCCGGATTTACTGGGCGTAAAGGATGCGTAGGCGG': ['gut2',
              'gut2',
              'gut2',
              'gut2'],
             'CTAG

Produce an ASV feature table looking something like this:

id       f1   f2   f3....etc

gut1  14   0  3...

gut2  0    10 15...